# Limpieza de Base de Datos

In [11]:
import os
import pandas as pd
import re

# Mostrar el directorio actual
print(os.getcwd())

# Cambiar el directorio de trabajo
os.chdir('/Users/matdknu/Dropbox/social-data-science/wiki-chile_project')



/Users/matdknu/Dropbox/social-data-science/wiki-chile_project


In [13]:
# Cargar la base de datos datos_wikipedia.csv
df_wikipedia = pd.read_csv('data/datos_wikipedia.csv')

# Mostrar las primeras filas del DataFrame
print(df_wikipedia.head())

              Nombre                                Fecha de nacimiento  \
0  Michelle Bachelet  29 de septiembre de 1951 (73 años)Santiago, Chile   
1       Ángela Jeria                 22 de agosto de 1926 Talca (Chile)   
2   Alberto Bachelet                27 de abril de 1923 Santiago, Chile   
3  Sebastián Dávalos  1 de junio de 1978 (46 años)Leipzig, República...   
4        Axel Kaiser      4 de julio de 1981  (43 años)Santiago (Chile)   

        Residencia       Nacionalidad  \
0         La Reina            Chilena   
1  Santiago, Chile            Chilena   
2              NaN            Chilena   
3  Santiago, Chile            Chilena   
4            Miami  Chilena y alemana   

                                              Cargos  \
0                              Médica; y; política;    
1                                       Arqueóloga;    
2      Político; ,; militar; y; piloto de aviación;    
3      Politólogo; y; político; Cohecho; y; estafa;    
4  Abogado; ,; escrito

In [21]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import re
from itertools import combinations

# Cargar el archivo
df = pd.read_csv('familias_chilenas.csv')  # Cambia ruta si es necesario

# Extraer nombres desde la columna "Familiares"
def extraer_nombres_familiares(texto):
    if not isinstance(texto, str) or texto.strip() == '{}':
        return []
    return re.findall(r"'([^']+)'\s*:", texto)

df["nombres_familiares"] = df["Familiares"].apply(extraer_nombres_familiares)
familiares_expandidos = df["nombres_familiares"].apply(pd.Series)
familiares_expandidos.columns = [f"familiar_{i+1}" for i in familiares_expandidos.columns]
df_resultado = pd.concat([df, familiares_expandidos], axis=1)

# Crear grafo y agregar relaciones familiares
G = nx.Graph()
edges_with_family = []

for idx, row in df_resultado.iterrows():
    miembro = row['Miembro']
    familia = row['Familia']
    
    for col in [c for c in df_resultado.columns if c.startswith("familiar_")]:
        familiar = row[col]
        if pd.notna(familiar):
            G.add_edge(miembro, familiar)
            edges_with_family.append((miembro, familiar, familia))

# NUEVO: conectar miembros de la misma familia entre sí
for familia, grupo in df_resultado.groupby("Familia"):
    miembros = grupo["Miembro"].dropna().unique()
    for m1, m2 in combinations(miembros, 2):
        G.add_edge(m1, m2)
        edges_with_family.append((m1, m2, familia))

# Posiciones para nodos
pos = nx.spring_layout(G, seed=42)

# Colores únicos por familia
familias = list(set(f for _, _, f in edges_with_family))
color_map = {fam: f"rgba({i*40%255},{(i*90)%255},{(i*130)%255},0.6)" for i, fam in enumerate(familias)}

# Aristas
edge_traces = []
for familia in familias:
    x_edges, y_edges = [], []
    for u, v, f in edges_with_family:
        if f == familia:
            x_edges += [pos[u][0], pos[v][0], None]
            y_edges += [pos[u][1], pos[v][1], None]
    edge_traces.append(
        go.Scatter(
            x=x_edges, y=y_edges,
            line=dict(width=1, color=color_map[familia]),
            hoverinfo='none',
            mode='lines',
            name=familia
        )
    )

# Nodos
node_x, node_y, node_text = [], [], []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=node_text,
    textposition="top center",
    hoverinfo='text',
    marker=dict(
        color='black',
        size=8,
        line_width=0.5
    )
)

# Graficar
fig = go.Figure(
    data=edge_traces + [node_trace],
    layout=go.Layout(
        title='Red de relaciones familiares (interactiva, agrupadas por familia)',
        titlefont_size=16,
        showlegend=True,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False)
    )
)

fig.show()


/var/folders/hg/s4skq8ks6g1fy2jw2fdkp0840000gn/T/ipykernel_30580/821032284.py:17: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [22]:
import pandas as pd
import networkx as nx
import re
from itertools import combinations

# -----------------------------
# 1. Cargar y limpiar datos
# -----------------------------
df = pd.read_csv('familias_chilenas.csv')  # Ajusta la ruta

# Extraer nombres desde la columna Familiares
def extraer_nombres_familiares(texto):
    if not isinstance(texto, str) or texto.strip() == '{}':
        return []
    return re.findall(r"'([^']+)'\s*:", texto)

df["nombres_familiares"] = df["Familiares"].apply(extraer_nombres_familiares)
familiares_expandidos = df["nombres_familiares"].apply(pd.Series)
familiares_expandidos.columns = [f"familiar_{i+1}" for i in familiares_expandidos.columns]
df_resultado = pd.concat([df, familiares_expandidos], axis=1)

# -----------------------------
# 2. Crear red solo entre familias distintas
# -----------------------------
# Mapeo: persona -> familia
persona_familia = df_resultado.set_index("Miembro")["Familia"].dropna().to_dict()

G = nx.Graph()

for idx, row in df_resultado.iterrows():
    miembro = row["Miembro"]
    familia_origen = row["Familia"]

    for col in [c for c in df_resultado.columns if c.startswith("familiar_")]:
        familiar = row[col]
        if pd.notna(familiar):
            # Verifica si el familiar está también en la base como Miembro
            familia_destino = persona_familia.get(familiar, None)
            if familia_destino and familia_destino != familia_origen:
                # Crear arista solo si conecta a otra familia
                G.add_edge(miembro, familiar, familia_origen=familia_origen, familia_destino=familia_destino)

# -----------------------------
# 3. Calcular métricas de red
# -----------------------------
# Centralidad
centralidad = nx.betweenness_centrality(G)

# Familias conectadas por nodo
conexiones_familia = {}
for node in G.nodes():
    familias = set()
    for vecino in G.neighbors(node):
        if vecino in persona_familia:
            familias.add(persona_familia[vecino])
    conexiones_familia[node] = len(familias)

# -----------------------------
# 4. Exportar tabla resumen
# -----------------------------
nodos_info = pd.DataFrame({
    "persona": list(G.nodes()),
    "familia": [persona_familia.get(n, "Desconocida") for n in G.nodes()],
    "centralidad": [centralidad[n] for n in G.nodes()],
    "familias_conectadas": [conexiones_familia[n] for n in G.nodes()]
})

# Ordenar por importancia
nodos_info = nodos_info.sort_values(by=["familias_conectadas", "centralidad"], ascending=False)

# Guardar si lo deseas
# nodos_info.to_csv("nodos_clave.csv", index=False)

print(nodos_info.head(10))

nodos_info

                         persona                 familia  centralidad  \
4        Francisco Antonio Pinto   Familia Pinto (Chile)     0.025966   
2                  Manuel Bulnes          Familia Prieto     0.018500   
7                  Felipe Bulnes          Familia Bulnes     0.008763   
11   Francisco Bulnes Sanfuentes      Familia Sanfuentes     0.005734   
30                 Germán Riesco       Familia Errázuriz     0.005518   
39     Ramón Subercaseaux Vicuña          Familia Vicuña     0.002921   
27  Federico Errázuriz Echaurren       Familia Errázuriz     0.006167   
64                Gabriel Valdés  Familia Valdés (Chile)     0.001298   
10       Jaime Bulnes Sanfuentes      Familia Sanfuentes     0.001190   
12      Manuel Bulnes Sanfuentes      Familia Sanfuentes     0.001190   

    familias_conectadas  
4                     5  
2                     3  
7                     3  
11                    3  
30                    3  
39                    3  
27            

/var/folders/hg/s4skq8ks6g1fy2jw2fdkp0840000gn/T/ipykernel_30580/2505289113.py:18: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [23]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import re

# -----------------------------
# 1. Cargar datos y limpiar
# -----------------------------
df = pd.read_csv('familias_chilenas.csv')

def extraer_nombres_familiares(texto):
    if not isinstance(texto, str) or texto.strip() == '{}':
        return []
    return re.findall(r"'([^']+)'\s*:", texto)

df["nombres_familiares"] = df["Familiares"].apply(extraer_nombres_familiares)
familiares_expandidos = df["nombres_familiares"].apply(pd.Series)
familiares_expandidos.columns = [f"familiar_{i+1}" for i in familiares_expandidos.columns]
df_resultado = pd.concat([df, familiares_expandidos], axis=1)

# Mapeo persona -> familia
persona_familia = df_resultado.set_index("Miembro")["Familia"].dropna().to_dict()

# -----------------------------
# 2. Crear red entre familias distintas
# -----------------------------
G = nx.Graph()
for idx, row in df_resultado.iterrows():
    miembro = row["Miembro"]
    familia_origen = row["Familia"]
    
    for col in [c for c in df_resultado.columns if c.startswith("familiar_")]:
        familiar = row[col]
        if pd.notna(familiar):
            familia_destino = persona_familia.get(familiar, None)
            if familia_destino and familia_destino != familia_origen:
                G.add_edge(miembro, familiar, familia_origen=familia_origen, familia_destino=familia_destino)

# -----------------------------
# 3. Preparar visualización
# -----------------------------
# Posiciones
pos = nx.spring_layout(G, seed=42)

# Colores únicos por familia
familias = list(set(nx.get_edge_attributes(G, 'familia_origen').values()))
color_map = {fam: f"rgba({i*40%255},{(i*90)%255},{(i*130)%255},0.7)" for i, fam in enumerate(familias)}

# Cantidad de familias distintas conectadas por nodo
familias_conectadas = {}
for node in G.nodes():
    familias_vecinas = set()
    for vecino in G.neighbors(node):
        fam = persona_familia.get(vecino)
        if fam:
            familias_vecinas.add(fam)
    familias_conectadas[node] = len(familias_vecinas)

# Aristas
edge_traces = []
for familia in familias:
    x_edges, y_edges = [], []
    for u, v, attrs in G.edges(data=True):
        if attrs.get("familia_origen") == familia:
            x_edges += [pos[u][0], pos[v][0], None]
            y_edges += [pos[u][1], pos[v][1], None]
    edge_traces.append(
        go.Scatter(
            x=x_edges, y=y_edges,
            line=dict(width=1, color=color_map[familia]),
            hoverinfo='none',
            mode='lines',
            name=familia
        )
    )

# Nodos
node_x, node_y, node_size, node_color, node_text = [], [], [], [], []
for node in G.nodes():
    x, y = pos[node]
    fam = persona_familia.get(node, "Desconocida")
    node_x.append(x)
    node_y.append(y)
    node_size.append(8 + familias_conectadas.get(node, 0) * 4)
    node_color.append(color_map.get(fam, "gray"))
    node_text.append(f"{node} ({fam})")

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=node_text,
    textposition="top center",
    hoverinfo='text',
    marker=dict(
        color=node_color,
        size=node_size,
        line_width=0.5
    )
)

# -----------------------------
# 4. Mostrar gráfico
# -----------------------------
fig = go.Figure(
    data=edge_traces + [node_trace],
    layout=go.Layout(
        title='Red de relaciones interfamiliares',
        titlefont_size=16,
        showlegend=True,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False)
    )
)

fig.show()


/var/folders/hg/s4skq8ks6g1fy2jw2fdkp0840000gn/T/ipykernel_30580/1473532998.py:17: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

